<figure>
  <IMG src="figures/logo-esi-sba.png" WIDTH=300 height="100" ALIGN="right">
</figure>

# Practical Trainining Series on Software Engineering For Data Science  
*By Dr. Belkacem KHALDI (b.khaldi@esi-sba.dz)*

# Notebook 9: Web Development for Data Science: Building Restful API with FastAPI (Part I)

The purpose of this [Jupyter Notebook] is to getting you familairized with Building Restful API with FastAPI For Data Science

#  FastAPI Hands-On

Re-test the codes seen in lecture 9 from slide 5 to 28.


In [27]:
from fastapi import FastAPI, Path, Query, Body, Form, File, UploadFile, Header, Request, Response, status, HTTPException
import nest_asyncio
import uvicorn
from enum import Enum
from typing import List
import json
import pandas as pd

# Allow re-entrant event loops in Jupyter
nest_asyncio.apply()

# Create FastAPI instance
app = FastAPI()

# Define a GET endpoint
@app.get("/")
async def hello_world():
    return {"hello": "world"}

class UserType(str, Enum):
    STANDARD = "standard"
    ADMIN = "admin"
    
@app.get("/users/{type}/{id}/")
async def get_user(type: UserType, id: int):
    return {"type": type, "id": id}


@app.get("/users/{id}")
async def get_user(id: int = Path(..., ge=1)):
    return {"id": id}

@app.get("/license-plates/{license}")
async def get_license_plate(license: str = Path(..., regex=r"^\d{5}-\d{3}-\d{2}$")):
    return {"license": license}

@app.get("/users")
async  def get_user(page: int = Query(1, gt = 0), size: int = Query(10, le = 100)):
    return {"page": page, "size": size}

@app.post("/users")
async def create_user(name: str = Body(...), age: int = Body(...)):
    return {"name": name, "age": age} 

@app.post("/createUser")
async def create_user(name: str = Form(...), age: int =  Form(...)):
    return {"name": name, "age": age} 

@app.post("/files")
async def upload_file(file: bytes = File(...)):
    return {"file_size": len(file)}

@app.get("/request")
async def get_request_object(request: Request):
    return {"path": request.url.path}

@app.post("/uploadFile")
async def upload_file(file: UploadFile = File(...)):
    return {"file_name": file.filename, "content_type": file.content_type}

@app.post("/uploadMultipleFiles")
async def upload_multiple_files(files: List[UploadFile]=File(...)):
    return [{"file_name": file.filename, "content_type": file.content_type}for file in files]

@app.get("/getHeader")
async def get_header(user_agent: str = Header(...)):
    return {"user_agent": user_agent}

@app.get("/setCookie")
async def custom_cookie(response: Response):
    response.set_cookie("cookie-name", "cookie-value", max_age=86400)
    return {"hello": "world"}

@app.post("/password")
async def check_password(password: str = Body(...), password_confirm: str = Body(...)):
    if password != password_confirm:
        raise HTTPException(
        status.HTTP_400_BAD_REQUEST,
        detail="Passwords don't match.",
        )
    return {"message": "Passwords match."}


from fastapi.templating import Jinja2Templates

templates = Jinja2Templates(directory="templates")

@app.get("/reply")
async def home(request: Request):
    return templates.TemplateResponse("/index.html",{"request":request})

@app.get("/houseprices")
async def home(request: Request):
    df = pd.read_csv("data/house_pricing.csv", nrows=25) 
    js = df.to_json(orient="records")
    data=json.loads(js)
    templates
    return templates.TemplateResponse("/houseprices.html",{"request":request, "house_prices":data})

# Run the application
uvicorn.run(app, host="127.0.0.1", port=8000)


C:\Users\asus\AppData\Local\Temp\ipykernel_18008\1889855951.py:34: DeprecationWarning: `regex` has been deprecated, please use `pattern` instead
  async def get_license_plate(license: str = Path(..., regex=r"^\d{5}-\d{3}-\d{2}$")):
INFO:     Started server process [18008]
INFO:     Waiting for application startup.
INFO:     Application startup complete.
INFO:     Uvicorn running on http://127.0.0.1:8000 (Press CTRL+C to quit)


INFO:     127.0.0.1:54662 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54662 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:54663 - "GET /docs/ HTTP/1.1" 307 Temporary Redirect
INFO:     127.0.0.1:54663 - "GET /docs HTTP/1.1" 200 OK
INFO:     127.0.0.1:54663 - "GET /openapi.json HTTP/1.1" 200 OK
INFO:     127.0.0.1:55723 - "GET /license-plates/11111-555-22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:56939 - "GET /license-plates/11111-555-22 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57242 - "GET /users/10 HTTP/1.1" 200 OK
INFO:     127.0.0.1:57719 - "POST /uploadMultipleFiles HTTP/1.1" 200 OK
INFO:     127.0.0.1:57909 - "GET /houseprices HTTP/1.1" 200 OK


INFO:     Shutting down
INFO:     Waiting for application shutdown.
INFO:     Application shutdown complete.
INFO:     Finished server process [18008]
